In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import generate_data
import numpy as np

In [24]:
name = 'protein'

In [ ]:
dataset = generate_data.generate_dataset(name, protein_data_path = "./../../../PT/protein_model/casp8/training_30", num_samples=6000)

Parsing ProteinNet:   0%|          | 0/458073 [00:00<?, ?line/s]

Parsed 13881 records


Frames+angles:   0%|          | 0/13881 [00:00<?, ?protein/s]

In [ ]:
data = dict()

In [ ]:
X_train, Y_train, X_test, Y_test = dataset[name]

val_ratio = 1/6
n = X_train.shape[0]
n_val = int(round(val_ratio * n))

rng = np.random.default_rng(0)
perm = rng.permutation(X_train.shape[0])
X_train, Y_train = X_train[perm], Y_train[perm]

X_val, Y_val = X_train[:n_val], Y_train[:n_val]
X_train2, Y_train2 = X_train[n_val:], Y_train[n_val:]

# 3) pack into dict (also fix the quote typo in X_test key)
data = {
    "X_train": X_train2.reshape(-1,16),
    "Y_train": Y_train2.reshape(-1,16),
    "X_val": X_val.reshape(-1,16),
    "Y_val": Y_val.reshape(-1,16),
    "X_test": X_test.reshape(-1,16),
    "Y_test": Y_test.reshape(-1,16),
}

In [ ]:
for k, v in data.items():
    try:
        print(f"{k}: {v.shape}")
    except AttributeError:
        print(f"{k}: {type(v)}")

In [ ]:
import torch

# save
save_path = name+"_dataset.pt"
torch.save(data, save_path)

# (optional) quick sanity-check load
loaded = torch.load(save_path, map_location="cpu", weights_only = False)
for k, v in loaded.items():
    try:
        print(k, v.shape)
    except AttributeError:
        print(k, type(v))

for name in ["X_train","Y_train","X_val","Y_val","X_test","Y_test"]:
    assert torch.isfinite(torch.tensor(loaded[name])).all(), f"Found NaN/Inf in {name}"